In [1]:
#%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### PEFT with LoRA (1 балл)

При обучении на более сложных задачах можно использовать адаптеры низкого ранга на основе статьи о LoRA.

Основная идея заключается в добавлении адаптеров низкого ранга параллельно существующим линейным слоям, как показано здесь:

<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>


В оригинальной статье по LoRA адаптеры добавлялись только к матрицам attention. Однако, последующие работы показывают, что полезно адаптировать также FFN. Но прежде чем приступить к обучению, нам нужно реализовать базовый слой LoRA.

In [2]:
# re-load the model to remove any previous PEFT tuners
model_name = 'Enoch/llama-7b-hf'

tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/home/lxuser/.pyenv/versions/nlp-mai/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/lxuser/.pyenv/versions/nlp-mai/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [3]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        module_outputs = self.module(input)
        lora_output = (input @ self.adapter_A) @ self.adapter_B
        
        return module_outputs + lora_output

In [4]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [5]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

In [6]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
batch = {k: v.to(device) for k, v in batch.items()}
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

/tmp/ipykernel_4215/1139030399.py:4: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float32):


Grad check successful, well done!


### (example) How to train your model

Чисто пример как учить модельку

Там основная таска ниже

In [7]:
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") # 32 lines
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        warmup_steps=250, max_steps=10, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()
# Это просто пример

/home/lxuser/.pyenv/versions/nlp-mai/lib/python3.10/site-packages/accelerate/accelerator.py:439: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/lxuser/.pyenv/versions/nlp-mai/lib/python3.10/site-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
1,1.895600
2,1.707000
3,0.893800
4,1.752200
5,1.168000
6,0.729600
7,1.519200
8,1.062900
9,0.667000
10,1.414500


TrainOutput(global_step=10, training_loss=1.280978512763977, metrics={'train_runtime': 4.2525, 'train_samples_per_second': 4.703, 'train_steps_per_second': 2.352, 'total_flos': 28253449027584.0, 'train_loss': 1.280978512763977, 'epoch': 0.62})


### Финальная задача: *реально* обучить модель (3 балла)

Ваша задача — дообучить модель для _генерации кода на Python_. Более конкретно:

* __датасет:__ используйте [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) или любой другой набор данных, содержащий Python-код. Так как для этой задачи не требуется много данных, будет достаточно использовать чистый `codeparrot`
* __короткие строки:__ возьмите первые 512 символов каждой строки
* __тип адаптера:__ используйте LoRA, как описано выше, __и, кроме того, хотя бы один из следующих элементов:__
   - дополнительный адаптер для lm_head
   - дополнительный адаптер для компонентов MLP (mlp.*)

* __обучение:__ вам не нужно обучать до сходимости. Если всё настроено правильно, модель должна начать `.generate` код уже после 500 шагов. Пожалуйста, используйте размер батча минимум 4 (4 x 1 x 512 токенов), при этом установите `gradient_accumulation_steps=4`.

Примечание: библиотека `peft` также имеет реализацию LoRA. Воспользуйтесь этим

__Альтернативное задание:__
Если не нравится такое задание - возьми другой датасет и дообучи на свою задачу. Заодно и напиши про результаты)
думаю, они будут очень приятны


In [8]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch']  # добавьте ещё несколько, которые не обязательно ассоциируются с Python

# <ОЧЕНЬ МНОГО ВАШЕГО КОДА>
# нужно подготовить некоторые примеры (или "выборки") для проверки вашего fine-tuned модели
# можно использовать transformers.Trainer (как выше) или свой код для тренировки
# после завершения тренировки покажите примеры текста, сгенерированного вашей моделью, предполагается, что это будут фрагменты кода на Python
# красиво выведите примеры генерации (рекомендация: используйте pandas или HTML) для удобного сравнения
# примечание: ваша модель с улучшением через LoRA может генерировать текст так же, как и не обученная модель (как выше)

In [9]:
model.eval()
gen_before = []

for prompt in prompts:
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    for i in tqdm(range(256)):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
    gen_before.append(tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))

gen_before = [i[3:] for i in gen_before]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

In [10]:
def preproc(samples):
    new_content = []
    for content in samples['content']:
        content = "\n".join([line[:512] for line in content.split("\n")])
        new_content.append(content)
    tokens_batch = tokenizer(new_content)
    new_tokens_batch = {"input_ids": [], 'attention_mask': []}
    
    for input_ids, attention_mask in zip(tokens_batch['input_ids'], tokens_batch['attention_mask']):
        new_tokens_batch['input_ids'].append(input_ids[:128])
        new_tokens_batch['attention_mask'].append(attention_mask[:128])

    return new_tokens_batch

In [11]:
import datasets


data = datasets.load_dataset("codeparrot/codeparrot-clean-valid")
data = data.map(lambda samples: preproc(samples), batched=True)
model._hf_peft_config_loaded = True

Repo card metadata block was not found. Setting CardData to empty.
/home/lxuser/.pyenv/versions/nlp-mai/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [12]:
trainer = transformers.Trainer(
    model=model, train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, gradient_accumulation_steps=4,
        warmup_steps=250, max_steps=100, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

Step,Training Loss
1,1.473500
2,1.319500
3,1.490100
4,1.277600
5,1.393400
6,1.507900
7,1.448200
8,1.313500
9,1.142900
10,1.430300


TrainOutput(global_step=100, training_loss=1.325284811258316, metrics={'train_runtime': 260.8515, 'train_samples_per_second': 6.134, 'train_steps_per_second': 0.383, 'total_flos': 8126834776473600.0, 'train_loss': 1.325284811258316, 'epoch': 0.03})

In [13]:
model.eval()
gen_after = []

for prompt in prompts:
    batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    for i in tqdm(range(256)):
        next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
        batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
        batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)
    gen_after.append(tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))
    
gen_after = [i[3:] for i in gen_after]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

  0%|          | 0/256 [00:00<?, ?it/s]

In [15]:

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

for prompt, gen_b, gen_a in zip(prompts, gen_before, gen_after):
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prompt, gen_b, gen_a))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
``,▶▶ The Best of the Best: 2018-2019 | The Best of the Best: 2018-2019 The Best of the Best: 2018-2019 The Best of the Best: 2018-2019 2018-09-20T15:00:00Z The Best of the Best: 2018-2019 The Best of the Best: 2018-2019 2018-09-20T15:00:00Z The Best of the Best: 2018-2019 The Best of the Best: 2018-2019 2018-09-20T15:00:00Z The Best of the Best: 2018-2019 2018-09-20T15:00:00Z The Best of the Best: 2018-2019 The Best of the,"# -*- coding: utf-8 -*- # # Copyright (C) 2015-2016, 2018-2019, 2020-2021, 2022-2023, 2024-2025, 2026-2027, 2028-2029, 2030-2031, 2032-2033, 2034-2035, 2036-2037, 2038-2039, 2040-2041, 2042-2043, 2044-2045, 2046-2047, 2048-2049, 2050-2051, 2052-2053, 2054-2055, 2056-2057, 205"
`import`,import Foundation public extension NSDate { public var isToday: Bool { return self.isEqualToDate(NSDate()) } public var isYesterday: Bool { return self.isEqualToDate(NSDate().addingDays(-1)) } public var isTomorrow: Bool { return self.isEqualToDate(NSDate().addingDays(1)) } public var isSameDayAs: Date { return self.isEqualToDate(NSDate()) } public var isSameDayAsDate: Date { return self.isEqualToDate(NSDate()) } public var isSameDayAsDate: Date { return self.isEqualToDate(NSDate()) } public var isSameDayAsDate: Date { return self.isEqualToDate(NSDate()) } public var isSameDayAsDate: Date { return self.isEqualToDate(NSDate()) },import os import sys import time import traceback from django.core.exceptions import ImproperlyConfigured from django.core.management import call_command from django.core.management.base import BaseCommand from django.core.management.base import CommandError from django.core.management.base import NoArgsCommand from django.core.management.base import OutputWrapper from django.core.management.base import SubCommand from django.core.management.base import UserDetailsCommand from django.core.management.base import get_commands_from_app from django.core.management.base import get_commands_from_app_labels from django.core.management.base import get_commands_from_app_labels_or_module from django.core.management.base import get_commands_from_app_labels_or_module_and_module from django.core.management.base import get_commands_from_app_labels_or_module_and_module_and_module from django.core.management.base import get_commands_from_app_labels_or_module_and_module
`from`,"from __future__ import absolute_import from __future__ import division from __future__ import print_function import argparse import os import sys import numpy as np from tensorflow.python import pywrap_tensorflow as tf def main(unused_args): """"""Main function for the TensorFlow distribution."""""" # Ensure that we are in a TensorFlow installation. if not tf.test.is_built_with_cuda(): raise ValueError('This script requires TensorFlow to be built ' 'with CUDA support.') # Ensure that we are in a TensorFlow installation. if not tf.test.is_built_with_cuda(): raise ValueError('This script requires TensorFlow to be built ' 'with CUDA support.') # Ensure that we are in a TensorFlow installation. if not tf.test.is_built_with_cuda(): raise ValueError('This script requires TensorFlow to be built ' 'with C","from __future__ import absolute_import import logging import os import re import sys import warnings from django.conf import settings from django.core.exceptions import ImproperlyConfigured from django.utils.importlib import import_module from django.utils.module import import_from_module from django.utils.translation import ugettext_lazy as _ from django_extensions.db.models import ( BaseModel, ModelState, ModelStateManager, ) from django_extensions.db.models.fields import ( AutoField, BigIntegerField, BooleanField, CharField, DateTimeField, EmailField, FileField, FloatField, GenericIPAddressField, IntegerField, NullBooleanField, PositiveIntegerField, PositiveSmallIntegerField, SlugField, TextField, TimeField, URLField, UUIDField, ) from django_extensions.db.models.fields.json import JSONField from django"
`while`,"while(1) whil

По-моему крутой результат должен выйти )

опиши забавные инсайды, которые получил после выполнения этого задания